In [ ]:
%sql
-- Populate concepts frontfill input table with works that need concept inference
-- Excludes works already processed (via LEFT ANTI JOIN to predicted output table)
CREATE OR REPLACE TABLE openalex.works.works_concepts_frontfill_input
SELECT
  xxhash64(concat_ws('|', title, abstract,
    primary_location.source.display_name,
    primary_location.source.type)) AS concept_key,
  id as work_id,
  title,
  abstract,
  primary_location.source.display_name AS journal,
  primary_location.source.type AS doc_type
FROM openalex.works.openalex_works_base w
LEFT ANTI JOIN openalex.works.openalex_works_concepts_predicted p
  ON xxhash64(concat_ws('|', w.title, w.abstract,
     w.primary_location.source.display_name,
     w.primary_location.source.type)) = p.concept_key
WHERE (concepts IS NULL OR size(concepts) = 0)
  AND title IS NOT NULL
  AND ((length(title) > 20 AND length(abstract) > 50)
       OR length(title) > 50
       OR length(abstract) > 150);

In [ ]:
%sql
OPTIMIZE openalex.works.works_concepts_frontfill_input ZORDER BY (concept_key);

In [ ]:
%sql
SELECT FORMAT_NUMBER(COUNT(*),0) as num_works_to_process
FROM openalex.works.works_concepts_frontfill_input